<a href="https://colab.research.google.com/github/2s2e/AppleProject/blob/main/Steven_Copy_of_ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install numpy

In [ ]:
import numpy as np
# Add any `import`s here.
# You may import anything from the Python Standard Library (auto-grader uses python 3.10).

In [ ]:
from collections import defaultdict

class NGramModel:
    # Special tokens
    START_TOKEN = "<start>"
    STOP_TOKEN = "<stop>"
    UNK_TOKEN = "<unk>"

    def __init__(self, n: int, laplace_smoothing: bool = False) -> None:
        """Initialize the N-gram model.

        Args:
            n (int): The 'n' in the N-gram model. E.g., n=3 constructs a trigram model. We will only test `n in (1, 2, 3)`.
            laplace_smoothing (bool, optional): Wether to use Laplace smoothing. Defaults to False.
        """
        self.n = n
        self.laplace_smoothing = laplace_smoothing



        # Initialize variables here as you see fit.
        self.documents = []
        self.vocab = defaultdict(int)
        self.ngrams = defaultdict(int)
        self.ngram_counts = defaultdict(int)
        self.unknown_words = set()
        self.total_tokens = 0;

    def fit(self, dataset: list[list[str]]) -> None:
        """Build the N-gram model from a dataset.

        The dataset consists of a list of documents, where
        each document is made up of a list of words followed by a STOP_TOKEN.

        Args:
            dataset (list[list[str]]): The training set.
        """

        #build the vocab, and add our start tokens
        for i in range(len(dataset)):
            self.documents.append(dataset[i])
            for j in range(len(dataset[i])):
                self.vocab[dataset[i][j]] += 1
            #self.documents[i].insert(0, NGramModel.START_TOKEN)

        #find unknown words
        self.vocab[NGramModel.UNK_TOKEN] = 0
        for word in self.vocab:
            if self.vocab[word] < 3:
                self.unknown_words.add(word)
                #replaceing the # of occurences of unknown word with unk
                self.vocab[NGramModel.UNK_TOKEN] += self.vocab[word]

        for unk in self.unknown_words:
            del self.vocab[unk]

        #convert tokens to unks
        for i in range(len(self.documents)):
            for j in range(len(self.documents[i])):
                if self.documents[i][j] in self.unknown_words:
                    self.documents[i][j] = NGramModel.UNK_TOKEN

        #build n-grams
        for i in range(len(self.documents)):
            doc_copy = self.documents[i].copy()
            #add starts
            for j in range(self.n - 1):
                doc_copy.insert(0, NGramModel.START_TOKEN)

            #build ngrams
            for k in range(self.n - 1, len(doc_copy)):
                ngram = tuple(doc_copy[k - self.n + 1:k + 1])
                self.ngrams[ngram] += 1


        self.total_tokens = sum(self.vocab.values())




        print(len(self.vocab))


    def get_probability(self, ngram: list[str]) -> float:
        """Calculate the probability of an N-gram.

        For example, for `n=3` (trigram model),
        `get_probability(['Hello', 'world', STOP_TOKEN])`
        should return p(x_3=STOP_TOKEN|x_1='Hello', x_2='world').

        You may assume the N-gram only contains words from the vocabulary plus special tokens.

        Args:
            ngram (list[str]): The N-gram as a list of words. Must be `n` long.

        Returns:
            float: The probability of the N-gram.
        """
        if self.n == 1:
          return self.vocab[ngram[0]] / self.total_tokens

        numerator = 0
        denomerator = 0
        ngrams_list = list(self.ngrams.keys())
        ngram = tuple(ngram)


        for i in range(len(self.ngrams)):
          target_ngram = ngrams_list[i]

          #exact match of the ngram
          if target_ngram == ngram:
            numerator = self.ngrams[target_ngram]
          #given is true
          if ngram[0:self.n - 1] == target_ngram[0:self.n - 1]:
            denomerator += self.ngrams[target_ngram]

        if self.laplace_smoothing:
          numerator += 1
          denomerator += len(self.vocab)

        return numerator / denomerator


       # raise NotImplementedError('To be implemented in 1.1.1')

    def get_perplexity(self, doc: list[str]) -> float:
        """Calculates the perplexity of a sequence.

        Args:
            doc (list[str]): A sequence of words.

        Returns:
            float: The perplexity of the sequence.
        """
        doc_ngrams = []

        #pad with start
        doc_copy = doc.copy()
        for j in range(self.n - 1):
            doc_copy.insert(0, NGramModel.START_TOKEN)

        #convert the doc into a list of ngrams
        for i in range(len(doc)):
          ngram = tuple(doc_copy[i:i + self.n])
          doc_ngrams.append(ngram)
          print(ngram)

        #add together
        log_p_s = 0
        for ngram in doc_ngrams:
          prob = self.get_probability(ngram)
          log_p_s += np.log(prob)

        return np.exp(-log_p_s / (len(doc)))


        #raise NotImplementedError('To be implemented in 1.1.2'

In [ ]:
# Read the training set, split the documents into lists of words, and adding a STOP_TOKEN to the end of each document.

with open('1b_benchmark.train.tokens', 'r', encoding='utf-8') as file:
    dataset = [line.split() + [NGramModel.STOP_TOKEN] for line in file]

In [ ]:
# This is what the first document looks like.

dataset[0]

['Having',
 'a',
 'little',
 'flexibility',
 'on',
 'that',
 'issue',
 'would',
 'go',
 'a',
 'long',
 'way',
 'to',
 'putting',
 'together',
 'a',
 'final',
 'package',
 '.',
 '<stop>']

### 1.1.1 Implement `fit` and `get_probability`

Implement the methods marked with "To be implemented in 1.1.1."
For now you can ignore `laplace_smoothing` (you will implement that in 1.1.3).

Once you are done, the below should work.
This is roughly how we will test your implementation, though we will use a different training set and a few more test cases.

In [ ]:
unigram = NGramModel(n=1)
unigram.fit(dataset)



26602


In [ ]:
p = unigram.get_probability(["a"])
print(p)
assert np.allclose(p, 0.019381910832735126)

0.019381910832735126


In [ ]:
trigram = NGramModel(n=3)
trigram.fit(dataset)

p = trigram.get_probability(["a", "little", "flexibility"])
assert np.allclose(p, 0.006060606060606061)

26601


### 1.1.2 Implement `get_perplexity`

Implement the methods marked with "To be implemented in 1.1.2."

Once you are done, the below should work.
At this point, your implementation likely can't handle sequences with unseen N-grams. You will fix that in the next part.

In [ ]:
unigram = NGramModel(n=1)
unigram.fit(dataset)

perplexity = unigram.get_perplexity(dataset[0])
print(perplexity)
assert np.allclose(perplexity, 924.3003136958376)

26601
('Having',)
('a',)
('little',)
('flexibility',)
('on',)
('that',)
('issue',)
('would',)
('go',)
('a',)
('long',)
('way',)
('to',)
('putting',)
('together',)
('a',)
('final',)
('package',)
('.',)
('<stop>',)
887.5482553542884


AssertionError: 

In [ ]:
trigram = NGramModel(n=3)
trigram.fit(dataset)

perplexity = trigram.get_perplexity(dataset[0])
assert np.allclose(perplexity, 8.42398352642198)

26601
('<start>', '<start>', 'Having')
('<start>', 'Having', 'a')
('Having', 'a', 'little')
('a', 'little', 'flexibility')
('little', 'flexibility', 'on')
('flexibility', 'on', 'that')
('on', 'that', 'issue')
('that', 'issue', 'would')
('issue', 'would', 'go')
('would', 'go', 'a')
('go', 'a', 'long')
('a', 'long', 'way')
('long', 'way', 'to')
('way', 'to', 'putting')
('to', 'putting', 'together')
('putting', 'together', 'a')
('together', 'a', 'final')
('a', 'final', 'package')
('final', 'package', '.')
('package', '.', '<stop>')


### 1.1.3 Implement Laplace Smoothing

Modify you implementation such that when `laplace_smoothing=True`,
`get_probability` and `get_perplexity` return probability and perplexity with Laplace smoothing.

After you are done, the below should work.

In [ ]:
trigram = NGramModel(n=3, laplace_smoothing=True)
trigram.fit(dataset)

# `["hello", "world", "!"]` does not exist in the training set,
# but with Laplace smoothing this should now have a small probability.
p = trigram.get_probability(["hello", "world", "!"])
assert np.allclose(p, 3.7591158559506806e-05)

26601


In [ ]:
trigram = NGramModel(n=3, laplace_smoothing=True)
trigram.fit(dataset)

# Without Laplace smoothing you can't compute perplexity for this sequence because of the unseen N-grams.
perplexity = trigram.get_perplexity(
    ["This", "sequence", "contains", "unseen", "trigrams", "!", NGramModel.STOP_TOKEN]
)
print(perplexity)
assert np.allclose(perplexity, 7711.325324297971)

26601
('<start>', '<start>', 'This')
('<start>', 'This', 'sequence')
('This', 'sequence', 'contains')
('sequence', 'contains', 'unseen')
('contains', 'unseen', 'trigrams')
('unseen', 'trigrams', '!')
('trigrams', '!', '<stop>')
3836.067510421336


AssertionError: 

### Write-up

Add any write-up question related code below.